Import Data

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
file = "env_data.json"

# Read
env_data = pd.read_json(file, dtype=False, lines=True)
env_data.head()

,arithmetic_mean,arithmetic_standard_dev,county_code,county_name,fifty_percentile,first_max_value,fourth_max_value,latitude,longitude,ninety_eight_percentile,...,ninety_percentile,parameter_name,second_max_value,seventy_five_percentile,site_num,state_code,state_name,ten_percentile,third_max_value,year
0,4.193443,7.078815,001,Ada,4.4,17.8,14.3,43.643055,-116.265718,16.7,...,13.5,Ambient Min Temperature,16.7,10.2,0021,16,Idaho,-5.1,14.4,2003
1,0.200000,0.134840,001,Ada,0.1,0.5,0.4,43.643055,-116.265718,0.5,...,0.4,Sample Flow Rate- CV,0.4,0.3,0021,16,Idaho,0.1,0.4,2002
2,0.156000,0.112101,001,Ada,0.1,0.5,0.3,43.643055,-116.265718,0.5,...,0.3,Sample Flow Rate- CV,0.4,0.1,0021,16,Idaho,0.1,0.3,2004
3,9.912000,9.961355,001,Ada,7.1,43.2,16.2,43.643055,-116.265718,43.2,...,18.9,PM2.5 - Local Conditions,36.6,8.5,0021,16,Idaho,3.2,18.9,2004
4,15.591304,14.466825,001,Ada,10.4,62.7,27.6,43.643055,-116.265718,62.7,...,33.1,PM2.5 - Local Conditions,40.8,21.1,0021,16,Idaho,4.6,33.1,2002


In [85]:
# Check for duplicates - seems like we have multiple values for the same year, state, county, and parameter
# Thoughts?
test=env_data.loc[(env_data['state_code']=='16') & 
              (env_data['county_code']=='001') & 
              (env_data['year']=='2004') &
              (env_data['parameter_name']=='Ozone')]
test.to_csv('test.csv')

In [77]:
# Assuming we are good on duplicates, the following cells clean up the data & change it from long to wide format

# Create fips
env_data["fips"] = env_data["state_code"] + env_data["county_code"]

# Commented this out; will join on year & fips instead of the key
# env_data["key"] = env_data["state_code"] + env_data["county_code"] + env_data["year"]

# Subset dataframe for the 4 parameters we care about
env_wanted=env_data.loc[(env_data['parameter_name'] == "PM2.5 - Local Conditions") |
    (env_data['parameter_name'] == "Ozone") |
    (env_data['parameter_name'] == "Carbon monoxide") |
    (env_data['parameter_name'] == "Nitrogen dioxide (NO2)")
]
env_wanted.head()

,arithmetic_mean,arithmetic_standard_dev,county_code,county_name,fifty_percentile,first_max_value,fourth_max_value,latitude,longitude,ninety_eight_percentile,...,second_max_value,seventy_five_percentile,site_num,state_code,state_name,ten_percentile,third_max_value,year,fips,key
3,9.912000,9.961355,001,Ada,7.1,43.2,16.2,43.643055,-116.265718,43.2,...,36.6,8.5,0021,16,Idaho,3.2,18.9,2004,16001,160012004
4,15.591304,14.466825,001,Ada,10.4,62.7,27.6,43.643055,-116.265718,62.7,...,40.8,21.1,0021,16,Idaho,4.6,33.1,2002,16001,160012002
10,15.591304,14.466825,001,Ada,10.4,62.7,27.6,43.643055,-116.265718,62.7,...,40.8,21.1,0021,16,Idaho,4.6,33.1,2002,16001,160012002
11,9.912000,9.961355,001,Ada,7.1,43.2,16.2,43.643055,-116.265718,43.2,...,36.6,8.5,0021,16,Idaho,3.2,18.9,2004,16001,160012004
13,9.912000,9.961355,001,Ada,7.1,43.2,16.2,43.643055,-116.265718,43.2,...,36.6,8.5,0021,16,Idaho,3.2,18.9,2004,16001,160012004


In [87]:
# Use the pivot table method to transpose the data
flipped=env_wanted.pivot_table(index = ['county_code', 'county_name','latitude','longitude','site_num','state_code','state_name','year','fips','key']
                , columns = ['parameter_name'], 
# The aggregate function shouldnt make a difference; it seems to change the results, however, I think due to duplicates
#                aggfunc='sum',
                               fill_value=0).reset_index()
flipped.head()

county_code county_name   latitude  longitude site_num  \
parameter_name                                                          
0                      001   Abbeville  34.325318 -82.386376     0001   
1                      001   Abbeville  34.325318 -82.386376     0001   
2                      001   Abbeville  34.325318 -82.386376     0001   
3                      001   Abbeville  34.325318 -82.386376     0001   
4                      001   Abbeville  34.325318 -82.386376     0001   

               state_code      state_name  year   fips        key  ...  \
parameter_name                                                     ...   
0                      45  South Carolina  1991  45001  450011991  ...   
1                      45  South Carolina  1992  45001  450011992  ...   
2                      45  South Carolina  1993  45001  450011993  ...   
3                      45  South Carolina  1994  45001  450011994  ...   
4                      45  South Carolina  1995  45001  450011995  ...   

               seventy_five_percentile                           \
parameter_name                   Ozone PM2.5 - Local Conditions   
0                                0.223                      0.0   
1                                0.234                      0.0   
2                                0.257                      0.0   
3                                0.233                      0.0   
4                                0.249                      0.0   

                ten_percentile                                \
parameter_name Carbon monoxide Nitrogen dioxide (NO2)  Ozone   
0                          0.0                    0.0  0.141   
1                          0.0                    0.0  0.158   
2                          0.0                    0.0  0.170   
3                          0.0                    0.0  0.157   
4                          0.0                    0.0  0.142   

                                        third_max_value  \
parameter_name PM2.5 - Local Conditions Carbon monoxide   
0                                   0.0             0.0   
1                                   0.0             0.0   
2                                   0.0             0.0   
3                                   0.0             0.0   
4                                   0.0             0.0   

                                                                       
parameter_name Nitrogen dioxide (NO2)  Ozone PM2.5 - Local Conditions  
0                                 0.0  0.287                      0.0  
1                                 0.0  0.338                      0.0  
2                                 0.0  0.312                      0.0  
3                                 0.0  0.340                      0.0  
4                                 0.0  0.348                      0.0  

[5 rows x 62 columns]

In [88]:
# Update columms names by combining measure name
flipped.columns = flipped.columns.map('|'.join).str.strip('|')
flipped.head()

,county_code,county_name,latitude,longitude,site_num,state_code,state_name,year,fips,key,...,seventy_five_percentile|Ozone,seventy_five_percentile|PM2.5 - Local Conditions,ten_percentile|Carbon monoxide,ten_percentile|Nitrogen dioxide (NO2),ten_percentile|Ozone,ten_percentile|PM2.5 - Local Conditions,third_max_value|Carbon monoxide,third_max_value|Nitrogen dioxide (NO2),third_max_value|Ozone,third_max_value|PM2.5 - Local Conditions
0,001,Abbeville,34.325318,-82.386376,0001,45,South Carolina,1991,45001,450011991,...,0.223,0.0,0.0,0.0,0.141,0.0,0.0,0.0,0.287,0.0
1,001,Abbeville,34.325318,-82.386376,0001,45,South Carolina,1992,45001,450011992,...,0.234,0.0,0.0,0.0,0.158,0.0,0.0,0.0,0.338,0.0
2,001,Abbeville,34.325318,-82.386376,0001,45,South Carolina,1993,45001,450011993,...,0.257,0.0,0.0,0.0,0.170,0.0,0.0,0.0,0.312,0.0
3,001,Abbeville,34.325318,-82.386376,0001,45,South Carolina,1994,45001,450011994,...,0.233,0.0,0.0,0.0,0.157,0.0,0.0,0.0,0.340,0.0
4,001,Abbeville,34.325318,-82.386376,0001,45,South Carolina,1995,45001,450011995,...,0.249,0.0,0.0,0.0,0.142,0.0,0.0,0.0,0.348,0.0


In [89]:
# Check for duplicates - on updated data
test2=flipped.loc[(flipped['state_code']=='16') & 
              (flipped['county_code']=='001') & 
              (flipped['year']=='2004') ]
test2.to_csv('test2.csv')